In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pylab as plt
import cv2
from matplotlib import image
import os
import numpy as np
from scipy import ndimage
from scipy.ndimage import morphology
import mach_functions


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
yamlFile="./Params.yaml"
Params = mach_functions.load_yaml(yamlFile)
Filters = mach_functions.readInfilters(Params)

In [3]:
Images = mach_functions.readInImages(Params)


In [4]:
detectedCells = dict()

In [7]:
# pick up the rod cells first

detectedCells['rod'] = mach_functions.Rodrotate(Images['grayImage'],\
                                   Filters['NormalFilters']['rod'],\
                                   Filters['PenaltyFilters']['rodp'],\
                                   rod_thres=Params['ThresValues']['rod'],\
                                   rodp_thres=Params['ThresValues']['rodp'])

In [10]:

newtestImage = mach_functions.removeDetectedCells(Images['grayImage'],detectedCells['rod'],bgmthres=0.4)

In [11]:
detectedCells['ram'], detectedCells['hyp'] = mach_functions.giveRamHyp(newtestImage,\
                                                 Filters['PartialFilters']['ramp'],\
                                                 Filters['PartialFilters']['hypp'],\
                                                 Params['ThresValues']['ramp'],\
                                                 Params['ThresValues']['hypp'] )

In [12]:
# remove the ramified and hypertrophic cells
newtestImage = mach_functions.removeDetectedCells(newtestImage,detectedCells['ram'],bgmthres=0.4)
newtestImage = mach_functions.removeDetectedCells(newtestImage,detectedCells['hyp'],bgmthres=0.4)

In [13]:
detectedCells['amoe'],detectedCells['dys'] = mach_functions.giveAmoeDys(newtestImage,Filters['NormalFilters']['amoe'],amoethres=0.33,areathes=210)

In [ ]:
machresults = mach_functions.machPerformance(detectedCells,Images['masks'])
machresults.resetTPFPWeights()
machresults.calculatePFscore()
machresults.plotTPFPfigure('newTPFP.png')
